# Les Optimisations de type réseau

## Introduction.
Les optimisations de type réseaux ont cela en commun qu'elles mettent en jeu 
des noeuds (nodes) et des arrêtes valuées (arcs).

Dans cette étude, je vais tenter, une fois les optimisations classiques abordées,
d'étudier la possibilité d'y ajouter des paramêtres non linéaires dans un second temps.

Etude globale proposée par Estelle Derrien - Github15000
 
## Sommaire

   - 1. Transports (Transportation problem)
         - Description
         - Modélisation mathématique
         - Un exemple avec Python Pulp
         - Un exemple avec Cplex et une tâche dummy.
   - 2. Transports avec relais (Transshipment problem)
         - Description
         - Modélisation mathématique
         - Un exemple avec Python Pulp
         - Un exemple avec Cplex et une tâche dummy.
   - 3. Affectation (Assignment problem)
         - Description
         - Modélisation mathématique
         - Exemples avec les solveurs
   - 4. Optimisation de production à l'aide d'un modèle de type réseau.
         - Description
         - Modélisation mathématique
         - Exemples avec les solveurs
   - 5. Plus court chemin (Shortest path problem)
         - Description
         - Modélisation mathématique
         - Exemples avec les solveurs
   - 6. Problème de Flots (Flow)
         - Description Flots maximum et Flots minimum
         - Modélisation mathématique
         - Exemples avec les solveurs
   - 7. Optimisation financière à l'aide d'un modèle de type réseau.
         - Description
         - Modélisation mathématique
         - Exemples avec les solveurs

## Sources 

Les livres suivants : 
 
- Operations research de Hillier
- Operations Management de R Russel
- Optimizations modeling with spreadsheets de R. Baker



# Transports (Transportation problem)

Le Problème de base :
On a n usines qui produisent un seul objet, on a m magasins, le coût de transport est différent pour chaque routes, les usines fournissent un nombre d'objets maximum, les magasins ont un besoin de stock minimum.
Comment minimiser le côut global de transport ?

## Modélisation Maths